In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)




# size of the window for data preparation
split_window_size = 20

# Initialize lists to store training and temporary sets
x_train_list, y_train_list, x_temp_list, y_temp_list = [], [], [], []

# Iterate through the data with the specified window size
for i in range(0, len(x_data) - split_window_size, split_window_size + 1):
    x_train_temp = x_data.iloc[i:i+split_window_size+1]
    y_train_temp = y_data.iloc[i:i+split_window_size+1]

    # Separate the last row for the temporary set
    x_train = x_train_temp.iloc[:-1]
    y_train = y_train_temp.iloc[:-1]

    x_temp = x_train_temp.iloc[-1:]
    y_temp = y_train_temp.iloc[-1:]

    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_temp_list.append(x_temp)
    y_temp_list.append(y_temp)

# Concatenate the lists into pandas DataFrames
x_train = pd.concat(x_train_list)
y_train = pd.concat(y_train_list)
x_temp = pd.concat(x_temp_list)
y_temp = pd.concat(y_temp_list)

# print(y_train.head(50))
x_temp_train, x_temp_val, y_temp_train, y_temp_val = train_test_split(x_temp, y_temp, test_size=0.2, random_state=42)


# Split x_temp and y_temp into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42)


"""
# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))
"""




scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)




"""
# x_train_tensor inverse

x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""



'\n# x_train_tensor inverse\n\nx_test_original = scaler.inverse_transform(x_train_tensor.numpy())\nprint("\nFirst row of x_test_original:")\nprint(x_test_original[0])\n\nprint("\nFirst row of x_train:")\nprint(x_train.head(1))\n\n\n\nprint("\nLast row of x_test_original:")\nprint(x_test_original[-1])\n\nprint("\nLast row of x_train:")\nprint(x_train.tail(1))\n'

In [14]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

In [15]:
import torch
import torch.nn as nn
import itertools



# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [8]
num_layers_list = [2]
learning_rates = [0.0011, 0.001, 0.0009]
window_sizes = [1]

num_epochs = 500
patience = 20  # Number of epochs to wait for improvement

# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Walk-forward validation training with sliding window for each hyperparameter combination
for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

        # Early stopping based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch}, validation loss: {val_loss}')
                break

        # Test the model
        test_loss = 0.0
        model.eval()
        with torch.no_grad():
            for i in range(len(x_test_tensor)):
                window_end = min(i + window_size, len(x_test_tensor))
                inputs = x_test_tensor[i:window_end].unsqueeze(0)
                labels = y_test_tensor[window_end - 1]

                outputs = model(inputs)
                test_loss += criterion(outputs, labels)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}, Test Loss: {test_loss / len(x_test_tensor)}')

    print(f'Final Test Loss: {test_loss / len(x_test_tensor)}')


Hyperparameters: input_size=7, hidden_size=8, num_layers=2, learning_rate=0.0011, window_size=1


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/500], Training Loss: 20343.56775972129, Validation Loss: 20772.96484375, Test Loss: 18378.20703125
Epoch [2/500], Training Loss: 15975.709533204068, Validation Loss: 18510.458984375, Test Loss: 16460.921875
Epoch [3/500], Training Loss: 12701.79457816684, Validation Loss: 13764.28125, Test Loss: 11978.0341796875
Epoch [4/500], Training Loss: 10496.436667058597, Validation Loss: 11254.5087890625, Test Loss: 9714.1279296875
Epoch [5/500], Training Loss: 8904.020835812396, Validation Loss: 9800.4111328125, Test Loss: 8429.302734375
Epoch [6/500], Training Loss: 7566.5063295737855, Validation Loss: 11722.322265625, Test Loss: 10213.236328125
Epoch [7/500], Training Loss: 8899.490160372994, Validation Loss: 10694.3330078125, Test Loss: 9186.943359375
Epoch [8/500], Training Loss: 5591.344159374201, Validation Loss: 7925.54052734375, Test Loss: 6865.27978515625
Epoch [9/500], Training Loss: 4684.559803394485, Validation Loss: 5554.3984375, Test Loss: 4653.3447265625
Epoch [10/500], 

In [16]:
ä

NameError: name 'ä' is not defined

In [ ]:
import torch
import torch.nn as nn
import itertools

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [8]
num_layers_list = [2]
learning_rates = [0.0011]
window_sizes = [1]

num_epochs = 500
patience = 20  # Number of epochs to wait for improvement

# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Function to create and train LSTM model with given hyperparameters
def train_lstm_model(input_size, hidden_size, num_layers, learning_rate, window_size, x_train_tensor, y_train_tensor, x_val_tensor, y_val_tensor):
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    no_improvement_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvement_counter = 0
        else:
            no_improvement_counter += 1

        if no_improvement_counter >= patience:
            print("Early stopping! Validation loss has not improved in the last {} epochs.".format(patience))
            break

    return model

# Function to evaluate the model on test data
def evaluate_model(model, x_test_tensor, y_test_tensor, window_size):
    test_loss = 0.0
    model.eval()

    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    return test_loss / len(x_test_tensor)

best_hyperparameters = None
best_test_loss = float('inf')

# Iterate over hyperparameter combinations and train models
for input_size, hidden_size, num_layers, learning_rate, window_size in hyperparameter_combinations:
    print(f"Training with hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    model = train_lstm_model(input_size, hidden_size, num_layers, learning_rate, window_size, x_train_tensor, y_train_tensor, x_val_tensor, y_val_tensor)
    test_loss = evaluate_model(model, x_test_tensor, y_test_tensor, window_size)

    print(f"Test Loss: {test_loss}")

    if test_loss < best_test_loss:
        best_test_loss = test_loss
        best_hyperparameters = (input_size, hidden_size, num_layers, learning_rate, window_size)

    if no_improvement_counter >= patience:
        print("Early stopping! Validation loss has not improved in the last {} epochs.".format(patience))
        break

print("Best Hyperparameters:")
print(f"input_size={best_hyperparameters[0]}, hidden_size={best_hyperparameters[1]}, num_layers={best_hyperparameters[2]}, learning_rate={best_hyperparameters[3]}, window_size={best_hyperparameters[4]}")


Training with hyperparameters: input_size=7, hidden_size=8, num_layers=2, learning_rate=0.0011, window_size=1


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/500], Training Loss: 20306.313814295034, Validation Loss: 20753.515625
Epoch [2/500], Training Loss: 16048.840531120357, Validation Loss: 18560.318359375
Epoch [3/500], Training Loss: 12729.05149925699, Validation Loss: 13655.046875
Epoch [4/500], Training Loss: 10512.95328698902, Validation Loss: 11163.7900390625
Epoch [5/500], Training Loss: 9026.591069200795, Validation Loss: 9856.130859375
Epoch [6/500], Training Loss: 13646.906094545182, Validation Loss: 10473.8955078125
Epoch [7/500], Training Loss: 7557.543504811881, Validation Loss: 11543.12109375
Epoch [8/500], Training Loss: 6201.650715865325, Validation Loss: 12255.1552734375
Epoch [9/500], Training Loss: 5044.054255530144, Validation Loss: 9074.212890625
Epoch [10/500], Training Loss: 4239.9348373606435, Validation Loss: 6048.8974609375
Epoch [11/500], Training Loss: 3588.590451966964, Validation Loss: 4562.951171875
Epoch [12/500], Training Loss: 3035.274334255149, Validation Loss: 3664.72119140625
Epoch [13/500],

NameError: name 'criterion' is not defined

In [ ]:
import torch
import torch.nn as nn
import itertools

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [8]
num_layers_list = [2]
learning_rates = [0.0015]
window_sizes = [1]

num_epochs = 500
patience = 20  # Number of epochs to wait for improvement

# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Function to create and train LSTM model with given hyperparameters
def train_lstm_model(input_size, hidden_size, num_layers, learning_rate, window_size, x_train_tensor, y_train_tensor, x_val_tensor, y_val_tensor):
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    no_improvement_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvement_counter = 0
        else:
            no_improvement_counter += 1

        if no_improvement_counter >= patience:
            print("Early stopping! Validation loss has not improved in the last {} epochs.".format(patience))
            break

    return model

# Function to evaluate the model on test data
def evaluate_model(model, x_test_tensor, y_test_tensor, window_size):
    test_loss = 0.0
    model.eval()

    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    return test_loss / len(x_test_tensor)

best_hyperparameters = None
best_test_loss = float('inf')

# Iterate over hyperparameter combinations and train models
for input_size, hidden_size, num_layers, learning_rate, window_size in hyperparameter_combinations:
    print(f"Training with hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    model = train_lstm_model(input_size, hidden_size, num_layers, learning_rate, window_size, x_train_tensor, y_train_tensor, x_val_tensor, y_val_tensor)
    test_loss = evaluate_model(model, x_test_tensor, y_test_tensor, window_size)

    print(f"Test Loss: {test_loss}")

    if test_loss < best_test_loss:
        best_test_loss = test_loss
        best_hyperparameters = (input_size, hidden_size, num_layers, learning_rate, window_size)

    if no_improvement_counter >= patience:
        print("Early stopping! Validation loss has not improved in the last {} epochs.".format(patience))
        break

print("Best Hyperparameters:")
print(f"input_size={best_hyperparameters[0]}, hidden_size={best_hyperparameters[1]}, num_layers={best_hyperparameters[2]}, learning_rate={best_hyperparameters[3]}, window_size={best_hyperparameters[4]}")


Training with hyperparameters: input_size=7, hidden_size=8, num_layers=2, learning_rate=0.0015, window_size=1


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/500], Training Loss: 18770.752336328915, Validation Loss: 19579.3359375
Epoch [2/500], Training Loss: 13740.519708273014, Validation Loss: 16766.568359375
Epoch [3/500], Training Loss: 10378.604738698396, Validation Loss: 11175.2041015625
Epoch [4/500], Training Loss: 8209.93255598469, Validation Loss: 9167.1962890625
Epoch [5/500], Training Loss: 13574.830220232927, Validation Loss: 10389.9453125
Epoch [6/500], Training Loss: 9492.488707595063, Validation Loss: 8144.0302734375
Epoch [7/500], Training Loss: 5810.206425079688, Validation Loss: 6087.51318359375
Epoch [8/500], Training Loss: 5467.8671059176595, Validation Loss: 5999.220703125
Epoch [9/500], Training Loss: 2988.2715256312536, Validation Loss: 4043.23828125
Epoch [10/500], Training Loss: 2367.885133884168, Validation Loss: 3482.7080078125
Epoch [11/500], Training Loss: 1921.6371661279293, Validation Loss: 3057.561279296875
Epoch [12/500], Training Loss: 1564.506560606458, Validation Loss: 2137.929931640625
Epoch [1

KeyboardInterrupt: 

In [ ]:
ä

In [ ]:
import torch
import torch.nn as nn
import itertools

# Define the hyperparameters to search over
input_sizes = [7]

hidden_sizes = [8]
num_layers_list = [2]
learning_rates = [0.001]
window_sizes = [1]

num_epochs = 500
patience = 10  # Number of epochs to wait for improvement

# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Function to create and train LSTM model with given hyperparameters
def train_lstm_model(input_size, hidden_size, num_layers, learning_rate, window_size, x_train_tensor, y_train_tensor, x_val_tensor, y_val_tensor):
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    no_improvement_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvement_counter = 0
        else:
            no_improvement_counter += 1

        if no_improvement_counter >= patience:
            print("Early stopping! Validation loss has not improved in the last {} epochs.".format(patience))
            break

    return model

# Function to evaluate the model on test data
def evaluate_model(model, x_test_tensor, y_test_tensor, window_size):
    test_loss = 0.0
    model.eval()

    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    return test_loss / len(x_test_tensor)

best_hyperparameters = None
best_test_loss = float('inf')

# Iterate over hyperparameter combinations and train models
for input_size, hidden_size, num_layers, learning_rate, window_size in hyperparameter_combinations:
    print(f"Training with hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    model = train_lstm_model(input_size, hidden_size, num_layers, learning_rate, window_size, x_train_tensor, y_train_tensor, x_val_tensor, y_val_tensor)
    test_loss = evaluate_model(model, x_test_tensor, y_test_tensor, window_size)

    print(f"Test Loss: {test_loss}")

    if test_loss < best_test_loss:
        best_test_loss = test_loss
        best_hyperparameters = (input_size, hidden_size, num_layers, learning_rate, window_size)

print("Best Hyperparameters:")
print(f"input_size={best_hyperparameters[0]}, hidden_size={best_hyperparameters[1]}, num_layers={best_hyperparameters[2]}, learning_rate={best_hyperparameters[3]}, window_size={best_hyperparameters[4]}")


Training with hyperparameters: input_size=7, hidden_size=8, num_layers=2, learning_rate=0.001, window_size=1


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/500], Training Loss: 20835.642038314178, Validation Loss: 21205.44140625
Epoch [2/500], Training Loss: 16828.648456067745, Validation Loss: 18972.677734375
Epoch [3/500], Training Loss: 13498.493071557354, Validation Loss: 14851.197265625
Epoch [4/500], Training Loss: 11239.545385571631, Validation Loss: 11871.763671875
Epoch [5/500], Training Loss: 9568.744962493936, Validation Loss: 10662.0888671875
Epoch [6/500], Training Loss: 23455.901811745836, Validation Loss: 22218.861328125
Epoch [7/500], Training Loss: 12995.45392429937, Validation Loss: 9855.7158203125
Epoch [8/500], Training Loss: 7643.12833366478, Validation Loss: 9261.47265625
Epoch [9/500], Training Loss: 6493.353465517648, Validation Loss: 17180.90234375
Epoch [10/500], Training Loss: 5612.694482069061, Validation Loss: 9053.916015625
Epoch [11/500], Training Loss: 4809.324778606503, Validation Loss: 6324.97802734375
Epoch [12/500], Training Loss: 4146.258019157851, Validation Loss: 5033.62255859375
Epoch [13/5

NameError: name 'criterion' is not defined